### Install RabbitMQ

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace rabbit

cat ./kubernetes/cicd/argocd/rabbit-secret.template.yml | sed "s/RABBITMQ_PASSWORD/$ALPHA_RABBITMQ_PASSWORD/" | kubectl create -n rabbit -f -

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the RabbitMQ Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/rabbit.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -


### Install Redis

In [ ]:
source ./config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace redis

kubectl config use-context ${CICD_CLUSTER}
cat ./kubernetes/cicd/argocd/redis.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -

### Install Wavefront Proxy

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create namespace wavefront
cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$ALPHA_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create namespace wavefront
cat ./kubernetes/cicd/argocd/wavefront-token.template.yml | sed "s/WAVEFRONT_TOKEN/$BRAVO_WAVEFRONT_TOKEN/" | kubectl create -n wavefront -f -

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`
BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the Wavefront Proxy Helm chart in Argo CD...'
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/wavefront-proxy.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

# Configure [Tanzu Service Mesh](https://console.cloud.vmware.com)

### Enable Istio automatic sidecar injection

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl label namespace default istio-injection=enabled --overwrite

- Onboard alpha & bravo clusters into Tanzu Service Mesh
- Create the "acme.com" global namespace and connect the default namespaces in the alpha & bravo clusters

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}
kubectl create -f ./kubernetes/istio/ingressgateway.yaml
kubectl create -f ./kubernetes/istio/virtualservice-alpha.yaml

echo
kubectl config use-context ${BRAVO_CLUSTER}
kubectl create -f ./kubernetes/istio/ingressgateway.yaml
kubectl create -f ./kubernetes/istio/virtualservice-bravo.yaml

## Configure GitOps for Alpha cluster

In [ ]:
source ./config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${ALPHA_CLUSTER}

# copy rabbit secrets to default namespace
kubectl get secret rabbit --namespace=rabbit --export -o yaml | kubectl apply --namespace=default -f -

# copy redis secrets to default namespace
kubectl create secret generic redis-alpha --namespace=default --from-literal="redis-password"=`kubectl get --namespace redis secret redis-alpha -o=jsonpath='{.data.redis-password}' | base64 -D`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the applications in Argo CD...'
cat ./kubernetes/cicd/argocd/account-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/confirmation-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -
cat ./kubernetes/cicd/argocd/payment-service.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/"  | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -

## Configure GitOps for Bravo cluster

In [ ]:
source ./config.sh

BRAVO_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${BRAVO_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the applications in Argo CD...'
cat ./kubernetes/cicd/argocd/dotnet-service.yml | sed "s/CLUSTER_URL/${BRAVO_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/bravo/" | kubectl apply -n argocd -f -

## Retrieve the external hostname for the Istio ingress gateway

In [ ]:
source ./config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo 'Alpha cluster ingress gateway:'
kubectl get service istio-ingressgateway -n istio-system -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'

echo
echo
kubectl config use-context ${BRAVO_CLUSTER}

echo 'Bravo cluster ingress gateway:'
kubectl get service istio-ingressgateway -n istio-system -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'

### To Do
- Add links to running applications
- Rework the readme file
- Use Harbor registry to demonstrate CVEs in public images
- Use [direnv](https://direnv.net/)?